In [24]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "college-information-llm"

In [2]:
llm = ChatOpenAI()

In [3]:
from knowledgebase import TXTKnowledgeBase

In [4]:
kb=TXTKnowledgeBase(txt_source_folder_path='lxbd')

In [4]:
#kb.initiate_documents()

=> TXT files loading and chunking done.
=> vector initialised


In [5]:
vector=kb.return_retriever_from_persistant_vector_db()

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [28]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever(search_kwargs={'k':4})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [32]:
response_stream=retrieval_chain.stream({"input": "美国博士申请成绩单认证应该怎么处理"})
output={}
curr_key = None
for chunk in response_stream:
    for key in chunk:
        if key not in output:
            output[key] = chunk[key]
        else:
            output[key] += chunk[key]
        if key != curr_key:
            print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
        else:
            print(chunk[key], end="", flush=True)
        curr_key = key



input: 美国博士申请成绩单认证应该怎么处理

context: [Document(page_content='一、什么是官方成绩单？\n\n官方成绩单是由学校开具的学生成绩证明，申请留学时需要的学校官方成绩单需要经过翻译且加盖学校公章。\n\n二、什么时候需要用官方成绩单？\n\n申请美国本科和申请美国研究生需要上交官方成绩单，成绩单可以体现学生的个人能力，美国高校将成绩单列为判断该学生能否被录取的重要材料之一。\n\n三、官方成绩单所包含的内容有哪些？\n\n1、学生个人信息\n\n学生姓名性别院系入学日期毕业日期学位专业等\n\n2、 课程信息\n\n课程名称课程属性（必修/选修）学分成绩标准GPA专业内部排名等信息\n\n注意：一般国内成绩体系为百分制，GPA为5.0，和美国成绩体制较为不同，所以申请美国高校时一般需要成绩认证。具体可以参见下方“成绩单认证”部分。\n\n四、学生需要去哪里开具官方成绩单？\n\n学生到学校教务处或行政机构开具中、英文成绩单。\n\n需注意的是，个别学校没有成绩单模板，或者没有英文成绩单，学生需提供中、英文成绩单模板给学校，并翻译好成绩单，然后用学校抬头纸打印，并盖好公章，之后装入学校官方信封，在封口处加盖公章。\n\n（图片来源浙江师范大学官网）\n\n五、开具官方成绩单的步骤？\n\n准备好所有的个人身份信息文件，包含身份证、户口本、毕业证书等。 填写开具成绩单的申请表或者口头描述自己需要中英文成绩单。包括开具成绩单的原因，需要的份数等。没有成绩单模板，或者没有英文成绩单的学校学生需提供翻译好的成绩单供学校核查。注意，中英文成绩单都需要用学校的抬头纸打印。缴费，国内成绩单开具需要付款，各学校成绩单价格不同，要收据或发票。 核对无误后，在成绩单上按学校规定加盖公章。将加盖公章的中英文成绩单装入学校官方信封，封口，并在封口处加盖公章。\n\n六、官方成绩单示例\n\n使用学校抬头纸（学校官方文件纸）打印中、英文成绩单。落款处有学校公章，成绩单需放入学校官方信封，封口，在封口处加盖学校公章。\n\n（高中成绩单示例）\n\n（大学成绩单示例） 七、怎样寄送官方成绩单到申请院校？ 1、地址 申请美国本科及研究生，需要寄送官方成绩单（Official Transcript）到学校指定的地址，通常指学校负责招生的部门，但也要

In [18]:
r

{'answer': ''}

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

#

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)


In [12]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="美国大学可以申请博士吗？"), AIMessage(content="可以！")]
response=retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "详细解释一下"
})
response['answer']

'在美国，大学本科学位持有者和研究生学位持有者都可以申请攻读博士学位。通常来说，本科申请博士的要求更高，需要具备出色的科研背景和学术成就。如果没有相关的研究经历和发表文章，通常更倾向于持有研究生学位的申请者。博士学位是美国教育体系中的最高学位，获得博士学位意味着已经在学术领域达到了高水平，具备独立从事科学研究工作的能力。博士学位持有者在就业市场上通常具有更多的机会和竞争力。'